# ParaView Hello Sphere Test
This notebook can connect to a parallel set of ParaView servers running on the allocated compute node.
It creates a synthetic data source (a sphere), and creates a polygonal display of it. Then, it creates a ParaView display widget showing the primary render view. The notebook further demonstrates how we may use interaction widgets (sliders), to change the resolution of the sphere.

In [1]:
from paraview.simple import *
from paraview.modules.vtkRemotingCore import vtkProcessModule

In [2]:
# to run in parallel on-the-allocated node, one would issue an srun command
# at the terminal:
# module load ParaView/5.8.0-CrayGNU-19.10-EGL
# srun -n 8 `which pvserver`
# followed by a Connect() command

Connect("localhost")

Connection (cs://localhost:11111) [2]

In [3]:
rank = vtkProcessModule.GetProcessModule().GetPartitionId()
nbprocs = servermanager.ActiveConnection.GetNumberOfDataPartitions()
info = GetOpenGLInformation(location=servermanager.vtkSMSession.RENDER_SERVER)
print("nbprocs= ",nbprocs)

nbprocs=  8


In [4]:
Version = str(GetParaViewVersion())
if rank == 0:
    print("ParaView Version ", Version)
    print("rank=", rank, "/", nbprocs)
    print("Vendor:   %s" % info.GetVendor())
    print("Version:  %s" % info.GetVersion())
    print("Renderer: %s" % info.GetRenderer())

ParaView Version  5.8
rank= 0 / 8
Vendor:   NVIDIA Corporation
Version:  4.6.0 NVIDIA 418.39
Renderer: Tesla P100-PCIE-16GB/PCIe/SSE2


In [5]:
sphere = Sphere(ThetaResolution=32, PhiResolution=32)

pidscal = ProcessIdScalars(sphere)

rep = Show(pidscal, GetActiveView())
ColorBy(rep, 'ProcessId')
processIdLUT = GetColorTransferFunction('ProcessId')
processIdLUT.AnnotationsInitialized = 1
processIdLUT.InterpretValuesAsCategories = 1

IndexedColors = [
    1.0, 1.0, 1.0,
    1.0, 0.0, 0.0,
    0.0, 1.0, 0.0,
    0.0, 0.0, 1.0,
    1.0, 1.0, 0.0,
    1.0, 0.0, 1.0,
    0.0, 1.0, 1.0,
    0.63, 0.63, 1.0,
    0.67, 0.5, 0.33,
    1.0, 0.5, 0.75,
    0.53, 0.35, 0.7,
    1.0, 0.75, 0.5]

a = []
for i in range(nbprocs):
    a.extend((str(i), str(i)))

processIdLUT.Annotations = a
processIdLUT.IndexedColors = IndexedColors
rep.Representation = "Surface With Edges"

In [6]:
from ipyparaview.widgets import PVDisplay
disp = PVDisplay(GetActiveView())
w = display(disp)

PVDisplay(resolution=(400, 400))

In [8]:
# Interact from ipywidgets offers us a simple way to interactively control values with a callback function
from ipywidgets import interact, IntSlider

# sets the Theta and Phi resolution values, triggers a pipeline update, and renders the result
def Sphere_resolution(res):
    sphere.ThetaResolution = sphere.PhiResolution = res
    sphere.UpdatePipeline()
    
interact(Sphere_resolution, res=IntSlider(min=3, max=48, step=1, value=12))
display(w)

interactive(children=(IntSlider(value=12, description='res', max=48, min=3), Output()), _dom_classes=('widget-…

None

We can cleanly close the connection with the following command

In [12]:
Disconnect()